In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm
from sklearn.neighbors import KDTree

In [2]:
# path to data
raw_data_path = "/home/ctchervenkov/Documents/data/ARE/are_npvm_referenz_2040_data/"
output_path = "/home/ctchervenkov/Documents/data/ARE/"
shapefile = "04-DTV_2040_MIV_Referenz__KGM_1250_reduziert/DTV_2040_MIV_Referenz__KGM_1250_reduziert_link.SHP";
public_transport_shapefile = "05-DTV_2040_OEV_Referenz_KGM_1250_reduziert/DTV_2040_OEV_Referenz__KGM_1250_reduziert_link.SHP";

In [3]:
# read private transport data
df_raw = gpd.read_file(
            "%s/%s" % (raw_data_path, shapefile),
            encoding = "latin1"
        ).to_crs({'init': 'EPSG:2056'})

In [4]:
# list of fields and renames
fields_direction_A = ['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 
                          'NUMLANES', 'CAPPRT', 'V0PRT', 'VOLVEHPR~1', 'VOLVEH_T~2', 'VOLVEH_T~3',
                          'VOLVEH_T~4', 'VOLVEH_T~5']

fields_direction_B = ['R_NO', 'R_FROMNO~6', 'R_TONODENO',
                          'R_TYPENO', 'R_TSYSSET', 'R_LENGTH', 'R_NUMLANES', 'R_CAPPRT',
                          'R_V0PRT', 'R_VOLVEH~7', 'R_VOLVEH~8', 'R_VOLVEH~9', 'R_VOLVE~10',
                          'R_VOLVE~11']

renames = {
    'NO' : 'link_id', 
    'FROMNODENO' : 'from_node', 
    'TONODENO' : 'to_node', 
    'TYPENO' : "link_type", 
    'TSYSSET' : "transit_system", 
    'LENGTH' : "link_length_km",
    'NUMLANES' : "number_lanes", 
    'CAPPRT' : "model_capacity", 
    'V0PRT' : "model_speed_km/h", 
    'VOLVEHPR~1' : 'total_load_veh/day', 
    'VOLVEH_T~2' : 'delivery_vehicles_veh/day', 
    'VOLVEH_T~3' : 'trucks_veh/day',
    'VOLVEH_T~4' : 'semi_trailers_veh/day', 
    'VOLVEH_T~5' : 'passenger_vehicles_veh/day', 
    'R_NO' : 'link_number',  
    'R_FROMNO~6' : 'from_node',
    'R_TONODENO' : 'to_node',
    'R_TYPENO' : "link_type", 
    'R_TSYSSET' : "transit_system", 
    'R_LENGTH' : "link_length_km", 
    'R_NUMLANES' : "number_lanes", 
    'R_CAPPRT' : "model_capacity",
    'R_V0PRT' : "model_speed_km/h", 
    'R_VOLVEH~7' : 'total_load_veh/day', 
    'R_VOLVEH~8' : 'delivery_vehicles_veh/day', 
    'R_VOLVEH~9' : 'trucks_veh/day', 
    'R_VOLVE~10' : 'semi_trailers_veh/day',
    'R_VOLVE~11' : 'passenger_vehicles_veh/day', 
    'geometry' : 'geometry',
}

In [5]:
# separate by direction
df_raw_direction_A = df_raw[fields_direction_A].rename(index=str, columns=renames)
df_raw_direction_B = df_raw[fields_direction_B].rename(index=str, columns=renames)

In [6]:
# clean data
df_raw_direction_A.loc[:,"direction"] = 0
df_raw_direction_B.loc[:,"direction"] = 1
df_raw_direction_A["link_length_km"] = df_raw_direction_A["link_length_km"].str.split("km").str.get(0).astype(float)
df_raw_direction_B["link_length_km"] = df_raw_direction_B["link_length_km"].str.split("km").str.get(0).astype(float)
df_raw_direction_A["model_speed_km/h"] = df_raw_direction_A["model_speed_km/h"].str.split("km/h").str.get(0).astype(int)
df_raw_direction_B["model_speed_km/h"] = df_raw_direction_B["model_speed_km/h"].str.split("km/h").str.get(0).astype(int)

In [7]:
# merge
df = pd.concat([df_raw_direction_A, df_raw_direction_B])

In [8]:
# compute travelled km
df.loc[:, "total_load_veh_km/day"] = df.loc[:, "total_load_veh/day"] * df.loc[:, "link_length_km"]
df.loc[:, "delivery_vehicles_veh_km/day"] = df.loc[:, "delivery_vehicles_veh/day"] * df.loc[:, "link_length_km"] 
df.loc[:, "trucks_veh_km/day"] = df.loc[:, "trucks_veh/day"] * df.loc[:, "link_length_km"] 
df.loc[:, "semi_trailers_veh_km/day"] = df.loc[:, "semi_trailers_veh/day"] * df.loc[:, "link_length_km"] 
df.loc[:, "passenger_vehicles_veh_km/day"] = df.loc[:, "passenger_vehicles_veh/day"] * df.loc[:, "link_length_km"] 

/home/ctchervenkov/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [9]:
df = df[["total_load_veh/day", "delivery_vehicles_veh/day", "trucks_veh/day", "semi_trailers_veh/day", "passenger_vehicles_veh/day",
         "total_load_veh_km/day", "delivery_vehicles_veh_km/day", "trucks_veh_km/day", "semi_trailers_veh_km/day", "passenger_vehicles_veh_km/day",
         "direction"]].groupby("direction").sum()

In [10]:
df.to_csv("%s/%s" % (output_path, "miv_statistics.csv"), sep=";")